In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline
import math
from keras.utils import np_utils

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (test_x, test_y) = cifar10.load_data()

num_classes = 10
y_train_cls = np_utils.to_categorical(y_train,num_classes)
test_y_cls = np_utils.to_categorical(test_y,num_classes)

print("Size of:")
print("- Training-set:\t\t{}".format(len(x_train)))
print("- Test-set:\t\t{}".format(len(test_x)))

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class_names = load_label_names()

def label_to_array(cls_true):
    true = cls_true.tolist()
    true_label = []
    for l in true:
        true_label += l
    return true_label
    

In [ ]:
#Data Dimensions

# CIFAR-10 images are 32 pixels in each dimension.
img_size = 32

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images:
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 10

In [ ]:
#Plotting Images

def plot_images(images, cls_true, cls_pred=None, smooth=True):

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing if we need to print ensemble and best-net.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Interpolation type.
        if smooth:
            interpolation = 'spline16'
        else:
            interpolation = 'nearest'

        # Plot image.
        ax.imshow(images[i, :, :, :],
                  interpolation=interpolation)
            
        # Name of the true class.
        cls_true_name = class_names[cls_true[i]]

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true_name)
        else:
            # Name of the predicted class.
            cls_pred_name = class_names[cls_pred[i]]

            xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
# Get the first 9 images from the test-set.
images = x_train[0:9]

# Get the true classes for those images.
cls_true = y_train[0:9]
cls_true = label_to_array(cls_true)

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true)


In [ ]:
# Normalise Data
x_train = (x_train.astype('float32'))/255.
test_x = (test_x.astype('float32'))/255.

In [ ]:
# Split data into train and validation sets
from sklearn.model_selection import train_test_split


train_x, validation_x, train_y, validation_y = train_test_split(x_train, y_train_cls,
                                                                test_size=0.2, random_state=42)

###  Import Required Functions

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def vgg(width, height, channels, architecture, classes):
    # Initialize the model along with the input shape to be
    # "channels last" ordering
    # Here, we are taking the size of all filters as 3x3 and adding BatchNormalisation after each layer
    # Architecture -- list telling architecture of VGG model (eg. for VGG11->[64,M,128,M,256,256,M,512,512,M])
    
    inputShape = (height, width, channels)
    
    inputs = Input(inputShape)
    net = inputs
    
    for i in (architecture):
        if i=='M':
            net = MaxPooling2D(pool_size = 2, strides = 2,
                          padding="same")(net)
        else:
            net = Conv2D(filters=i, kernel_size=(3,3),
                         dtype=tf.float32, strides=1,
                         padding='same')(net)
            net = Activation("relu")(net)
            net = BatchNormalization()(net)
        
    # Flatten
    net = Flatten()(net)
    
    # First Dense Layer with ReLU activation
    net = Dense(512)(net)
    net = Activation("relu")(net)
    net = Dropout(0.4)(net)

    # Final Dense Layer with softmax activation
    net = Dense(num_classes)(net)
    net = Activation("softmax")(net)
    
    model = Model(inputs=inputs, outputs=net, name='VGG')
    # return the constructed network architecture
    return model


In [ ]:
# VGG Models
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [ ]:
## Create Model
model = vgg(32, 32, 3, cfg['VGG11'], num_classes)

In [ ]:
model.summary()

In [ ]:
num_epochs = 100

# Initialize the optimizer and compile the model
optimizer = Adam(lr=0.1, decay=0.0005 / num_epochs)
print("Training network...")
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
             metrics=['accuracy'])


### Data Augmnetation

In [ ]:
# Use Data Augmentation or not
data_augmentation = True

In [ ]:
# Run training, with or without data augmentation.
if not augment_data:
    print('Not using data augmentation.')
    h = model.fit(train_x,
                  train_y,
                  batch_size = 128,
                  validation_data=(validation_x,validation_y),
                  epochs = num_epochs,
                  verbose = 1)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        fill_mode='nearest',
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(train_x,train_y, batch_size=128),
                        epochs=num_epochs, verbose=1, workers=4, 
                        validation_data=(validation_x,validation_y))

In [ ]:
#Evaluate on Test Set
result = model.evaluate(test_x, test_y_cls)

In [ ]:
#Predict on test_set
pred_y = model.predict(test_x)
pred_cls = np.argmax(pred_y, axis=1)


In [ ]:
#Some predicted Images
# Get the first 9 images from the test-set.
images = test_x[0:9]

# Get the true classes for those images.
cls_true = test_y[0:9]
cls_true = label_to_array(cls_true)

#Get Predicted Classes
cls_pred = pred_cls[0:9]
cls_pred = cls_pred.tolist()

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true, cls_pred)